## Process Calculated GPP

In this notebook, previously calculated GPP data NIRv is further processed to retrieve spatially aggregated monthly mean values for Germany and federal states.

In [1]:
# import libraries
import os
import glob
import xarray as xr
import pandas as pd
import geopandas as gpd
import rasterio
import odc.stac
import pathlib
from pathlib import Path
import warnings

In [2]:
# set working directory
os.chdir("E:/Master/Thesis/3_Data")
print("Current working directory: {0}".format(os.getcwd()))

Current working directory: E:\Master\Thesis\3_Data


In [3]:
warnings.filterwarnings("ignore", category = UserWarning)
warnings.filterwarnings("ignore", category = RuntimeWarning)

In [4]:
# spatially aggregate monthly GPP data
# to get mean values for Germany and federal states

In [5]:
# first, get values for broadleaved forest
# load all monthly gpp values as one dataset
directory = f"./GPP/GPP_NIRv/Monthly"
filelist = glob.glob(os.path.join(directory, "*BL*.nc"))
gpp_monthly = xr.open_mfdataset(filelist, concat_dim="time", combine="nested", engine="netcdf4")

In [6]:
# calculate mean gpp values per month for Germany
gpp_monthly_spatial_means = gpp_monthly.mean(['lon','lat'], skipna=True).to_dataframe()
gpp_monthly_spatial_means

,GPP,crs,spatial_ref
time,,,
1981-06-01,3.637814,0,0
1981-07-01,2.871488,0,0
1981-08-01,1.466914,0,0
1981-09-01,1.669887,0,0
1981-10-01,0.398454,0,0
...,...,...,...
2024-08-01,6.019095,-127,0
2024-09-01,5.164019,-127,0
2024-10-01,2.972307,-127,0


In [7]:
# drop unneeded columns
gpp_monthly_spatial_means = gpp_monthly_spatial_means.drop(['crs', 'spatial_ref'], axis=1)

In [8]:
# rename column with gpp value to show area
gpp_monthly_spatial_means.rename({"GPP":"GER"}, axis=1, inplace=True)

In [9]:
# now, calculate monthly mean gpp values for all federal states
# get list of shapefiles of all states
directory = "./Administrative_Borders"
files = list(pathlib.Path(directory).glob('*.shp'))

# loop over filelist to process each state
for f in range(len(files)):
    file = files[f]

    # load state shapefile 
    state = str(file)[23:25]
    state_shp = gpd.read_file(file)

    # reproject shapefile to match netcdf data
    state_shp = state_shp.to_crs(4326)

    # extract one date to create mask of state with dimensions of data
    gpp_ex = gpp_monthly.sel(time = "2000-01-01", method = "nearest")

    # create mask of research area using the dimensions of the exemplary data
    state_mask = rasterio.features.geometry_mask(state_shp.geometry, 
                                                out_shape=gpp_ex.odc.geobox.shape,
                                                transform=gpp_ex.odc.geobox.affine,
                                                all_touched=False,
                                                invert=False)
    
    state_mask = xr.DataArray(state_mask, dims=("lat", "lon"))

    # mask data
    gpp_state = gpp_monthly["GPP"].where(~state_mask)

    # calculate monthly mean gpp of state and extract to dataframe
    gpp_state_spatial_means = gpp_state.mean(['lon','lat'], skipna=True).to_dataframe()

    # drop unneeded columns
    gpp_state_spatial_means = gpp_state_spatial_means.drop(['crs', 'spatial_ref'], axis=1)

    # add monthly mean gpp values of state to dataframe
    gpp_monthly_spatial_means[state] = gpp_state_spatial_means["GPP"]   

In [10]:
gpp_monthly_spatial_means

,GER,BB,BE,BW,BY,HB,HE,HH,MV,NI,NW,RP,SH,SL,SN,ST,TH
time,,,,,,,,,,,,,,,,,
1981-06-01,3.637814,1.779946,NaN,3.898517,3.983941,NaN,2.683502,NaN,0.505244,3.672723,2.130469,NaN,NaN,NaN,2.799109,3.261831,3.210209
1981-07-01,2.871488,1.397233,0.137020,3.330665,3.313796,2.982336,3.292463,1.735976,2.509912,2.201916,2.690854,3.106560,1.820035,3.027926,1.061366,1.413533,2.738950
1981-08-01,1.466914,1.087284,0.052600,2.041835,1.802850,3.604556,1.331064,0.759020,1.275687,1.075457,0.966639,1.484000,1.420477,2.156466,0.730132,0.786608,1.157972
1981-09-01,1.669887,0.381774,-0.393742,1.978680,1.870598,2.794828,1.971938,0.531825,0.701810,1.335734,1.734870,2.261431,0.866130,1.977428,0.413913,0.611776,1.448737
1981-10-01,0.398454,-0.687918,-1.651820,1.813218,1.019412,0.259593,0.410732,-0.753764,-0.239120,-0.654029,-0.201852,-0.347364,-0.277773,1.564905,-0.425813,-0.965123,-0.052035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-01,6.019095,5.776002,4.728904,5.636565,5.814160,5.617053,6.173118,5.245027,6.107111,6.863436,6.611741,5.628627,6.235826,5.835349,4.395249,5.797201,6.221916
2024-09-01,5.164019,5.103750,4.571241,4.707010,4.917290,4.321928,5.141759,5.323732,5.455825,5.691698,5.425132,5.079465,5.255440,5.556856,4.332573,5.376019,5.507658
2024-10-01,2.972307,3.867199,3.384862,2.315018,2.636477,3.649596,2.732770,3.048760,3.809529,3.614861,3.127455,2.582118,3.719911,2.899941,3.043469,3.473473,3.140361


In [11]:
# save as csv dataset
gpp_monthly_spatial_means.to_csv("./Tables/Spatial_Means/GPP_NIRv_Monthly_Spatial_Means_BL.csv")

In [12]:
# now follow the same steps for needleleaved forest
# load all monthly gpp values as one dataset
directory = f"./GPP/GPP_NIRv/Monthly"
filelist = glob.glob(os.path.join(directory, "*NL*.nc"))
gpp_monthly = xr.open_mfdataset(filelist, concat_dim="time", combine="nested", engine="netcdf4")

In [13]:
# calculate mean gpp values per month for Germany
gpp_monthly_spatial_means = gpp_monthly.mean(['lon','lat'], skipna=True).to_dataframe()
gpp_monthly_spatial_means

,GPP,crs,spatial_ref
time,,,
1981-06-01,4.118287,0,0
1981-07-01,2.669756,0,0
1981-08-01,1.080396,0,0
1981-09-01,0.870074,0,0
1981-10-01,-0.747892,0,0
...,...,...,...
2024-08-01,6.243868,-127,0
2024-09-01,5.519111,-127,0
2024-10-01,3.273199,-127,0


In [14]:
# drop unneeded columns
gpp_monthly_spatial_means = gpp_monthly_spatial_means.drop(['crs', 'spatial_ref'], axis=1)

In [15]:
# rename column with gpp value to show area
gpp_monthly_spatial_means.rename({"GPP":"GER"}, axis=1, inplace=True)

In [16]:
# now, calculate monthly mean gpp values for all federal states
# get list of shapefiles of all states
directory = "./Administrative_Borders"
files = list(pathlib.Path(directory).glob('*.shp'))

# loop over filelist to process each state
for f in range(len(files)):
    file = files[f]

    # load state shapefile 
    state = str(file)[23:25]
    state_shp = gpd.read_file(file)

    # reproject shapefile to match netcdf data
    state_shp = state_shp.to_crs(4326)

    # extract one date to create mask of state with dimensions of data
    gpp_ex = gpp_monthly.sel(time = "2000-01-01", method = "nearest")

    # create mask of research area using the dimensions of the exemplary data
    state_mask = rasterio.features.geometry_mask(state_shp.geometry, 
                                                out_shape=gpp_ex.odc.geobox.shape,
                                                transform=gpp_ex.odc.geobox.affine,
                                                all_touched=False,
                                                invert=False)
    
    state_mask = xr.DataArray(state_mask, dims=("lat", "lon"))

    # mask data
    gpp_state = gpp_monthly["GPP"].where(~state_mask)

    # calculate monthly mean gpp of state and extract to dataframe
    gpp_state_spatial_means = gpp_state.mean(['lon','lat'], skipna=True).to_dataframe()

    # drop unneeded columns
    gpp_state_spatial_means = gpp_state_spatial_means.drop(['crs', 'spatial_ref'], axis=1)

    # add monthly mean gpp values of state to dataframe
    gpp_monthly_spatial_means[state] = gpp_state_spatial_means["GPP"]   

In [17]:
gpp_monthly_spatial_means

,GER,BB,BE,BW,BY,HB,HE,HH,MV,NI,NW,RP,SH,SL,SN,ST,TH
time,,,,,,,,,,,,,,,,,
1981-06-01,4.118287,1.451539,NaN,6.485208,5.075558,NaN,3.742689,NaN,3.128078,4.810337,NaN,3.593975,NaN,NaN,2.542818,2.666722,3.396694
1981-07-01,2.669756,0.729119,-0.601199,4.347859,4.136165,3.683605,4.170012,0.698194,2.214143,2.093985,4.411393,4.487583,3.102823,4.651948,0.639301,1.419169,2.382191
1981-08-01,1.080396,0.274825,-0.886415,2.751397,1.639071,4.220320,1.131361,0.095382,0.763905,1.621236,1.404869,1.226909,2.816672,2.675759,0.005446,0.327074,0.514120
1981-09-01,0.870074,-0.767158,-1.826410,2.344512,1.540159,1.849610,1.993972,-0.487064,-0.275379,1.130043,2.315127,2.638599,0.794493,2.516062,-0.467576,-0.484332,0.654259
1981-10-01,-0.747892,-2.207645,-3.922823,2.209815,0.199960,-0.422139,-0.003504,-2.711627,-2.393787,-1.930041,-0.414204,-1.016500,-1.750960,1.626996,-1.396020,-2.666367,-1.440088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-01,6.243868,5.727725,5.014842,6.778337,5.624643,5.897285,7.560306,5.874493,6.962389,7.570001,8.443568,6.920378,7.725540,6.729509,4.887180,5.848569,5.012962
2024-09-01,5.519111,5.717772,4.727754,4.882497,4.946016,4.244633,5.953420,5.775921,6.370820,6.177942,6.757596,6.536101,6.195267,6.619406,4.598336,6.034968,4.930594
2024-10-01,3.273199,4.365738,3.829041,2.375809,2.401871,2.316442,2.848150,2.689150,4.614805,3.847738,3.082079,2.693953,4.849759,2.557920,2.920075,4.308546,1.855874


In [18]:
# save as csv dataset
gpp_monthly_spatial_means.to_csv("./Tables/Spatial_Means/GPP_NIRv_Monthly_Spatial_Means_NL.csv")